# Install and Import all Libraries

In [ ]:
# installing pathway bokeh
!pip install pathway bokeh --quiet

In [ ]:
# suppress specific user warnings from Bokeh

import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='bokeh')

In [ ]:
# core Libraries

import numpy as np
import pandas as pd
import datetime
from datetime import datetime

In [ ]:
# Pathway & Bokeh

import pathway as pw
import bokeh.plotting
import panel as pn

# Load and Preprocess Parking Dataset

In [ ]:
# load the dataset

df = pd.read_csv('/content/dataset.csv')
df.head()

## Checking missing values

In [ ]:
# check for missing values

df.isna().sum()

In [ ]:
# combine date and time columns into a single datetime column

df['Timestamp'] = pd.to_datetime(
    df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
    format='%d-%m-%Y %H:%M:%S'
)

## Sort record by timestamps

In [ ]:
# sort records by timestamp

df = df.sort_values('Timestamp').reset_index(drop=True)

## Computing Occupancy Rates

In [ ]:
# compute occupancy rate

df["OccupancyRate"] = df["Occupancy"] / df["Capacity"]

In [ ]:
# preview processed data

df.head()

# Export Streamable Subset for Pathway

In [ ]:
# save selected columns to a CSV to simulate a data stream

df[["SystemCodeNumber","Timestamp", "Occupancy", "Capacity", "OccupancyRate"]].to_csv("parking_stream.csv", index=False)

# Define Schema and Load Streaming Data with Pathway

In [ ]:
# define schema for Pathway ingestion

class ParkingSchema(pw.Schema):
    SystemCodeNumber: str
    Timestamp: str
    Occupancy: int
    Capacity: int
    OccupancyRate: float

In [ ]:
# load data with simulated stream rate

data = pw.demo.replay_csv("parking_stream.csv", schema=ParkingSchema, input_rate=100)

# Convert Time Columns and Create Daily Lot Identifiers

In [ ]:
fmt = "%Y-%m-%d %H:%M:%S"

data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%d"),
    OccupancyRate = data.Occupancy / data.Capacity,
    day_id = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%d") + "_" + data.SystemCodeNumber,
)

In [ ]:
# ccreate lookup table of unique lot-day combinations

id_lookup = (
    data_with_time
    .select(pw.this.day_id, pw.this.SystemCodeNumber)
    .groupby(pw.this.day_id)
    .reduce(
        day_id = pw.reducers.min(pw.this.day_id),
        SystemCodeNumber = pw.reducers.min(pw.this.SystemCodeNumber)
    )
)

# Aggregate Daily Occupancy Stats and Compute Dynamic Price

In [ ]:
from datetime import timedelta

# daily rolling window aggregation

reduced = (
    data_with_time.windowby(
        pw.this.t,
        instance=pw.this.day_id,
        window=pw.temporal.tumbling(timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t = pw.this._pw_window_end,
        occ_max = pw.reducers.max(pw.this.OccupancyRate),
        occ_min = pw.reducers.min(pw.this.OccupancyRate),
        cap = pw.reducers.max(pw.this.Capacity),
        day_id = pw.reducers.min(pw.this.day_id),
        lot = pw.reducers.min(pw.this.SystemCodeNumber)
    )
)

# pricing formula: base price + scaled volatility

delta_window = reduced.with_columns(
    price = 10 + (pw.this.occ_max - pw.this.occ_min) / pw.this.cap
)

# Live Streaming Plot using Panel and Bokeh

In [ ]:
pn.extension()

# define interactive Bokeh plot


def price_plotter(source):
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Pathway: Daily Parking Price",
        x_axis_type="datetime",
    )

    fig.line("t", "price", source=source, line_width=2, color="navy")
    fig.scatter("t", "price", source=source, size=6, color="red", marker="circle")

    fig.xaxis.axis_label = "Time"
    fig.yaxis.axis_label = "Price ($)"
    return fig

# launch panel dashboard

viz = delta_window.plot(price_plotter, sorting_col="t")
pn.Column(viz).servable()
viz.servable()

# Export Real-Time Price Output to JSON Lines

In [ ]:
# write output to file for post-processing and visualization

pw.io.jsonlines.write(
    delta_window.select(
        pw.this.t,
        pw.this.lot,
        pw.this.price
    ),
    "pricing_output.jsonl"
)

# Run Pathway app

pw.run()


# Post-Run Visualization by Lot (Multiple Plots)

In [ ]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.palettes import Category10

output_notebook()

In [ ]:
# read saved prices

df_price = pd.read_json("pricing_output.jsonl", lines=True)
df_price["t"] = pd.to_datetime(df_price["t"])
df_price["lot"] = df_price["lot"].astype(str)

plots = []
lots = sorted(df_price["lot"].unique())
palette = Category10[10] + Category10[4]

for i, lot in enumerate(lots):
    df_lot = df_price[df_price["lot"] == lot].copy()
    source = ColumnDataSource(df_lot[["t", "price", "lot"]])

    fig = figure(
        height=300,
        width=800,
        title=f"Price Trend - Lot {lot}",
        x_axis_type="datetime",
        tools="pan,wheel_zoom,box_zoom,reset"
    )

    fig.line("t", "price", source=source, color=palette[i % len(palette)], line_width=2)
    fig.scatter("t", "price", source=source, size=5, marker="circle", color="black")

    hover = HoverTool(
        tooltips=[
            ("Lot", "@lot"),
            ("Time", "@t{%F %T}"),
            ("Price", "@price{$0.00}")
        ],
        formatters={"@t": "datetime"},
        mode="vline"
    )
    fig.add_tools(hover)

    fig.xaxis.axis_label = "Time"
    fig.yaxis.axis_label = "Price ($)"

    plots.append(fig)

show(column(*plots))

# Final Summary Plot: All Lots Combined

In [ ]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.palettes import Category10

output_notebook()

# normalize datetime precision

df = pd.read_json("pricing_output.jsonl", lines=True)
df["t"] = pd.to_datetime(df["t"]).dt.tz_localize(None)
df["t"] = df["t"].astype("datetime64[ms]")
df["lot"] = df["lot"].astype(str)

In [ ]:
# create combined figure

fig = figure(
    height=500,
    width=900,
    title="Daily Pricing Across Parking Lots",
    x_axis_type="datetime"
)

palette = Category10[10] + Category10[4]
lots = sorted(df["lot"].unique())

for i, lot in enumerate(lots):
    df_lot = df[df["lot"] == lot].copy()
    source = ColumnDataSource(df_lot[["t", "price", "lot"]])

    fig.line("t", "price", source=source, legend_label=f"Lot {lot}",
             color=palette[i % len(palette)], line_width=2)

In [ ]:
#add interactive hover tool

hover = HoverTool(
    tooltips=[
        ("Lot", "@lot"),
        ("Time", "@t{%F %T}"),
        ("Price", "@price{$0.00}")
    ],
    formatters={"@t": "datetime"},
    mode="vline"
)
fig.add_tools(hover)

fig.xaxis.axis_label = "Time"
fig.yaxis.axis_label = "Price ($)"
fig.legend.label_text_font_size = "8pt"

In [ ]:
#show plot

show(fig)